In [1]:
import os
import ub_dep_utils as ubdutil
import ub_dep_plots as ubdplt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# this script uses historical & simulated data on
# lees ferry natural flows and upper basin demands
# to estimate unregulated inflows into powell
# code calculates in 4 steps:
# 1. historical regression (1963 - 2020 and 1963 - 1988/ 1989-2020)
# 2. linear regression w/ simulated data
# 3. multivariate regression w/ simulated data
# 4. multivariate logistic regression w/ simulated data
# 5. UB demand scenarios are taken from the 6 web tool scenarios

fig_path = 'all_demands/'
ifolder = 'data/'
i2 = ''
historical_split_date = 1989 # year of UB depeletion step change in historical record
simulation_year_range = [2027, 2056] # range of simulation data w/ different UB demand scenarios
simulation_traces = 400 # number of hydrology traces in simulation data
error_range = [-2000000, 2000000] # plotting range for prediction errors

In [2]:
# Calculate regressions w/ simulated data
# read simulated data
print('get crss data')
natural_flows, powell_inflows, tot_demands, unreg_inflows, natural_flows_monthly, powell_inflows_monthly, tot_demands_monthly, unreg_inflows_monthly = ubdutil.make_regression_inputs(simulation_year_range, simulation_traces, i2)


get crss data


In [3]:
# multivariate logistic regression
print('....logistic')
param_table = pd.DataFrame()
predictions_lr = {}
errors_lr = {}
rsquared_vals_lr = {}
for monthNum in range(0, 12):
  predictions_lr[monthNum], errors_lr[monthNum], rsquared_vals_lr[monthNum], params = ubdutil.fit_logistic_regression(natural_flows_monthly[:,monthNum], tot_demands_monthly[:,monthNum], unreg_inflows_monthly[:,monthNum])
  print(monthNum)
  print(params)
  param_table.loc[monthNum, 'Magnitude1'] = float(params[0])
  param_table.loc[monthNum, 'Location1'] = float(params[3])
  param_table.loc[monthNum, 'Scale1'] = float(params[4])
  param_table.loc[monthNum, 'Magnitude2'] = float(params[5])
  param_table.loc[monthNum, 'Location2'] = float(params[1])
  param_table.loc[monthNum, 'Scale2'] = float(params[2])
param_table.to_csv(os.path.join(fig_path,'monthly_params.csv'))

....logistic
0
[ 0.77749195  8.64303899  0.52738475 -4.56311765  0.60960172  0.79620981]
1
[  0.56517744   8.5226012    0.344094   -13.41357072   0.19038349
   0.14063896]
2
[ 0.05822549 -0.13256506  2.87449368 -2.32406738 -5.52422775  0.01209586]
3
[ 0.08996839 -0.36676919  1.07227736  0.21089685 -6.14044423  0.0347166 ]
4
[ 7.58823684e-01 -6.91294245e+00 -4.92296384e+00  1.82794247e-01
 -8.15661058e-01  2.34752160e-09]
5
[ 1.14727836e+00 -7.41731465e+00 -1.39782659e+01  6.67207261e-01
 -7.05768073e-01  2.12894835e-05]
6
[ 3.50417839  0.51440299  1.29868085 -6.79739754  4.85191719  0.1976803 ]
7
[ 0.23809113  0.40061795  3.01229331 -5.7046322  -3.77186147  0.07969664]
8
[ 0.10872834 -0.02140345  2.70648365 -2.21239407 -5.84414626  0.05387157]
9
[ 0.04777327 -0.12222788  3.28327219 -1.67414847 -7.17930449  0.0276638 ]
10
[ 1.39983148 -0.045664    1.70678094 -4.0494225   0.98164557  0.01486523]
11
[ 1.00809412  0.27020711  2.90614392 -1.26879779  2.24973172  0.01820278]


In [4]:
param_table

,Magnitude1,Location1,Scale1,Magnitude2,Location2,Scale2
0,0.777492,-4.563118,0.609602,7.962098e-01,8.643039,0.527385
1,0.565177,-13.413571,0.190383,1.406390e-01,8.522601,0.344094
2,0.058225,-2.324067,-5.524228,1.209586e-02,-0.132565,2.874494
3,0.089968,0.210897,-6.140444,3.471660e-02,-0.366769,1.072277
4,0.758824,0.182794,-0.815661,2.347522e-09,-6.912942,-4.922964
5,1.147278,0.667207,-0.705768,2.128948e-05,-7.417315,-13.978266
6,3.504178,-6.797398,4.851917,1.976803e-01,0.514403,1.298681
7,0.238091,-5.704632,-3.771861,7.969664e-02,0.400618,3.012293
8,0.108728,-2.212394,-5.844146,5.387157e-02,-0.021403,2.706484
9,0.047773,-1.674148,-7.179304,2.766380e-02,-0.122228,3.283272


In [6]:
# Annualize the monthly predictions & errors for plotting
lr_predictions_annual = pd.DataFrame(predictions_lr).sum(axis=1)
errors_lr_annual = pd.DataFrame(errors_lr).sum(axis=1)
avg_monthly_rsqrd = pd.DataFrame(rsquared_vals_lr).mean(axis=1)

In [7]:
# Plot multivariate logistic regression
print('plot logistic regression')
ubdplt.plot_logistic_regression(natural_flows, tot_demands, unreg_inflows, lr_predictions_annual, avg_monthly_rsqrd, fig_path)
ubdplt.plot_logistic_error(natural_flows, tot_demands, unreg_inflows, errors_lr_annual, error_range, fig_path)

plot logistic regression
